In [1]:
#import libraries
import sys
import os
import time
import datetime as dt
import numpy as np
import pandas as pd
import norgatedata as ng
from functions import trend_filter, position_size_atr, signal_filter, select_positions_1, \
                      exposure_summary, process_universe_factors, filter_ranks, step_activation, \
                      construct_trade_df, read_prior_activations
from ib_connect import main as positions

sys.path.append(os.path.abspath('../fin_data'))
from trade_logic.indicators import stock_beta, stock_volatility, atr, ewma
from utils.date_functions import last_business_day, time_elapsed
from utils.postgresql_data_query import get_bars, get_bm_bars, get_effective_dates
from data_updater.norgate_data import NorgateData

start_time = time.time()

[2024-11-12 14:33:57.898863] WARNING: Norgate Data: Unable to obtain valid status from Norgate Data - perhaps the app Norgate Data Updater (NDU) has not been started on this PC?
[2024-11-12 14:33:59.698837] INFO: Norgate Data: NorgateData package v1.0.74: Init complete
connected to: dbmaster


In [2]:
## set up parameters
bm_col = '$SPX'
lo_rank = 50
hi_rank = 50
w_offset = 0
d_offset = max(1, w_offset * 7)
beta_n = 36
vola_n = 20
atr_n = 20
ewma_stop = 128
ewma_n = 128
dgt = 2
cols = ['Date','Open','High','Low','Close']
factor_cols = ['ticker','factor_date','factor_definition_id','rank']
model_factors = [85, 86, 107, 7]
activation_factor = 85
activation_threshold = 95
activation_mult = 2
risk_factor = 0.001
sector_max = 0.20
max_positions = 200

excluded_tickers = ['VZIO']

client_id = 1
account_number = "DU3208934"
path = '/Users/VadimKovshov/Dropbox/INVESTMENTS/EVALUTE/STOCKS/MODEL_OUTPUTS/MOMENTUM_MODEL/'

use_postgresql = True
end = last_business_day(offset=d_offset)
start = last_business_day(offset=d_offset + 400)
# current_date = dt.date.today()
# prior_model_date = '2024-09-16'
print(f'Historical range: {end.strftime("%Y-%m-%d")} to {start.strftime("%Y-%m-%d")}')

Historical range: 2024-11-11 to 2023-10-06


In [3]:
# get most recent positions from IB
positions, account_value = positions(client_id, account_number, path)

if positions.empty:
    print('No current positions found')
else:
    print(f'Current positions: {len(positions)}')
    positions.head(3)

if account_value:
    print(f'Account value: {account_value}')
else:
    account_value = 603538.23
    print(f'Using nominal account value: {account_value}')

position_tickers = positions['ticker'].tolist()
positions.head(3)

Current positions: 18
Account value: 650415.32


,account,ticker,position,average_cost
0,DU3208934,TRGP,160.0,167.776298
1,DU3208934,TRU,230.0,103.796570
2,DU3208934,ENVA,460.0,81.466790


In [4]:
# get universe factors for a given week
eff_date, pr_date = get_effective_dates(offset_0=w_offset)
model_date = eff_date.strftime("%Y-%m-%d")
prior_model_date = pr_date.strftime("%Y-%m-%d")

print(f'Factor dates now: {eff_date.strftime("%Y-%m-%d")}, {pr_date.strftime("%Y-%m-%d")}')

ranks_now = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                     hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols)

positions_now = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                     hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols, tickers=position_tickers)

ranks_now = pd.concat([ranks_now, positions_now])
ranks_now.drop_duplicates(subset=['ticker'], inplace=True)

# get universe factors for one week prior
eff_date, pr_date = get_effective_dates(offset_0=w_offset+1)
print(f'Factor dates prior: {eff_date.strftime("%Y-%m-%d")}, {pr_date.strftime("%Y-%m-%d")}')

ranks_prior = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                       hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols)

positions_prior = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                       hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols, tickers=position_tickers)

ranks_prior = pd.concat([ranks_prior, positions_prior])
ranks_prior.drop_duplicates(subset=['ticker'], inplace=True)

Factor dates now: 2024-11-08, 2024-11-01
Test universe: 1748
Test universe: 18
Factor dates prior: 2024-11-01, 2024-10-25
Test universe: 1703
Test universe: 18


In [5]:
# Obtain prior model output
prior_output = f'{path}momo_model_output_{prior_model_date}.csv'
prior_activations_df = read_prior_activations(prior_output)
# Filter the DataFrames
filter_used = 1
df_now = filter_ranks(ranks_now, filter=filter_used)
df_prior = filter_ranks(ranks_prior, filter=filter_used)

# Merge with prior activations to include the prior activation values
df_prior = pd.merge(df_prior, prior_activations_df, on='ticker', how='left')

combined_df = pd.merge(df_now, df_prior, on='ticker', how='left', suffixes=('', '_prior'), indicator=True)
# Assign labels based on the merge result
combined_df['label'] = combined_df['_merge'].map({'left_only': 'new', 'both': 'old'})

# Check if ticker is in position_tickers, and assign 'position' label
combined_df['label'] = np.where(combined_df['ticker'].isin(position_tickers), 'position', combined_df['label'])

# Handle missing activation_value or prior signal
combined_df['activation_value'] = combined_df['activation_value'].fillna(1)  # Set a default activation value if missing
combined_df[f'f_{activation_factor}_prior'] = combined_df[f'f_{activation_factor}_prior'].fillna(0)  # Default for prior signals

# Calculate the activation signal  
combined_df['activation'] = combined_df.apply(
    lambda row: step_activation(
        signal_now=row[f'f_{activation_factor}'], 
        signal_prior=row[f'f_{activation_factor}_prior'] if pd.notnull(row[f'f_{activation_factor}_prior']) else 0,
        activation_value=row['activation_value'],
        label=row['label'],
        extreme_threshold=activation_threshold, 
        activation_mult=activation_mult
    ), axis=1
)

combined_df = combined_df.drop(columns=['_merge'])
columns_to_keep = list(df_now.columns) + ['label', 'activation']
combined_df = combined_df[columns_to_keep]

# Optionally, reset the index
combined_df = combined_df.reset_index(drop=True)

# Filter tickers and limit to max_positions
tickers = (
    combined_df
    .loc[~combined_df['ticker'].isin(excluded_tickers), 'ticker']
    .unique()
    .tolist()[:max_positions]
)

print(f'Filtered tickers: {len(tickers)}')

Filtered tickers: 79


In [7]:
# get industry data
ng_obj = NorgateData(symbols= tickers, database='US Stocks')
ng_obj.process_metadata()
industry_df = ng_obj.df_result['metadata'].rename(columns={'symbol':'ticker', 'security_name':'name', 'GICS_level_1': 'sector', 'GICS_level_2': 'industry_group',
                                                            'GICS_level_3': 'industry', 'GICS_level_4': 'sub_industry'}).reset_index(drop=True)
industry_df = industry_df[['ticker', 'sector', 'industry_group', 'industry', 'sub_industry', 'name']]

sorted_df = pd.merge(combined_df, industry_df, on='ticker', how='left')

print (f'Sector tickers: {len(tickers)}')

Sector tickers: 79


In [8]:
# get benchmark data
if use_postgresql:
    bm_df = get_bm_bars(bm_col,start,end) # uses saved PostgreSQL data
else:
    bm_df = ng_obj.daily_bars('$SPX', start, end, interval='D') # uses NorgateData data
    bm_df = bm_df[cols]
    bm_df['Ticker'] = bm_col

print(f'Benchmark data: {len(bm_df)}')

Benchmark data: 273


In [9]:
exposure = trend_filter(bm_df, _c='Close')
exposure = 1 #exposure[-1]
print(f'Exposure: {exposure}')

Exposure: 1


In [10]:
# get stock data and process signals
df_f = pd.DataFrame()
i = 0
for t in tickers:
    i += 1
    try:
        if use_postgresql:
            st_df = get_bars([t],start,end,backtest=False) # uses saved PostgreSQL data
        else:
            st_df = ng_obj.daily_bars(t, start, end, interval='D') # uses NorgateData data
            st_df = st_df[cols]
            st_df['Ticker'] = t

        beta_value = round(stock_beta(df=st_df, _c='Close', bm_df=bm_df, bm='Close', n=beta_n),dgt)
        vola_value = stock_volatility(df=st_df, _c='Close', n=vola_n, annualized=False)
        atr_value = round(atr(df=st_df, _h='High', _l='Low', _c='Close', n=atr_n).iloc[-1],2)
        activation_value = combined_df.loc[combined_df['ticker'] == t, 'activation'].values[0]
        ewma_value = round(ewma(df=st_df, _c='Close', _n=(ewma_stop / activation_value), _mp=(ewma_stop / activation_value)).iloc[-1],dgt)
        last_close = st_df['Close'].iloc[-1]
        signal_value = signal_filter(df=st_df, _h='High', _l='Low', _c='Close', n=ewma_n, pct_change_threshold=0.15, period=90)
        atr_position_size = activation_value * signal_value * position_size_atr(account_value, risk_factor, exposure, atr_value) // 10 * 10
        dollar_position = atr_position_size * st_df['Close'].iloc[-1]
        pct_position = round(dollar_position / account_value,dgt)

        data = {
            'date': st_df['Date'].iloc[-1],
            'ticker': t,
            'beta': beta_value,
            'volatility': vola_value,
            'atr': atr_value,
            'last_close': last_close,
            'ewma_stop': ewma_value,
            'signal': signal_value,
            'new_position': atr_position_size,
            'dollar_position': dollar_position,
            'pct_position': pct_position,
            'activation_value': activation_value
        }
        df = pd.DataFrame([data])
        df_f = pd.concat([df_f,df],ignore_index=True)
        if i % 20 == 0:
            print(f'Completed: {i}, {t}, {time_elapsed(start_time)}')
    except Exception as e:
        print(f"Error processing {t}: {e}")  # Print out the exception for debugging
        continue

print(f'Completed: {i}, {t}, {time_elapsed(start_time)}')

Completed: 20, QFIN, 2 minutes and 31 seconds
Completed: 40, ENVA, 2 minutes and 33 seconds
Completed: 60, GRBK, 2 minutes and 35 seconds
Completed: 79, NVR, 2 minutes and 37 seconds


In [11]:
# final filter
df_f_final = pd.merge(sorted_df, df_f, on='ticker', how='left')
df_f_final = df_f_final[df_f_final['signal'] == 1].reset_index(drop=True).copy()
df_f_final = df_f_final.drop(columns=['factor_date']).reset_index(drop=True)

In [12]:
# create model output and trade dataframes
selected_positions_df = select_positions_1(df_f_final, max_exposure=exposure, max_sector_allocation=sector_max).sort_values(by='ticker')
trade_df = construct_trade_df(positions, selected_positions_df).sort_values(by='ticker')

In [13]:
summary_positions = selected_positions_df[selected_positions_df['new_position'] != 0]
styled_summary = exposure_summary(summary_positions)

print(f'Completed: {time_elapsed(start_time)}')
styled_summary

Total exposure: 0.94


Completed: 2 minutes and 37 seconds


,sector,sector_exp,industry,industry_exp,ticker,exposure,name
0,Energy,0.18,"Oil, Gas & Consumable Fuels",0.18,VNOM,0.08,Viper Energy Inc Class A Common
1,Energy,0.18,"Oil, Gas & Consumable Fuels",0.18,KMI,0.05,Kinder Morgan Inc Common
2,Energy,0.18,"Oil, Gas & Consumable Fuels",0.18,CNX,0.03,CNX Resources Corp Common
3,Energy,0.18,"Oil, Gas & Consumable Fuels",0.18,TPL,0.02,Texas Pacific Land Corp Common
4,Financials,0.18,Capital Markets,0.08,JEF,0.08,Jefferies Financial Group Inc Common
5,Financials,0.18,Consumer Finance,0.06,ENVA,0.06,Enova International Inc Common
6,Financials,0.18,Banks,0.04,MTB,0.04,M&T Bank Corp Common
7,Real Estate,0.18,Office REITs,0.07,HIW,0.04,Highwoods Properties Common
8,Real Estate,0.18,Office REITs,0.07,SLG,0.03,SL Green Realty Common
9,Real Estate,0.18,Retail REITs,0.05,AKR,0.05,Acadia Realty Trust Common


In [ ]:
# save output to csv
selected_positions_df.to_csv(f'{path}momo_model_output_{model_date}.csv', index=False)
trade_df.to_csv(f'{path}momo_trade_file_{model_date}.csv', index=False)
trade_df

,ticker,label,position,shares_to_trade,new_position,average_cost,dollar_position,pct_position,ewma_stop,name,sector
21,ACIW,position,570.0,-130.0,440.0,49.215842,24560.8,0.04,45.59,ACI Worldwide Inc Common,Information Technology
4,AKR,old,0.0,1250.0,1250.0,NaN,31475.0,0.05,21.58,Acadia Realty Trust Common,Real Estate
14,ALSN,position,280.0,-30.0,250.0,107.205048,30002.5,0.05,90.21,Allison Transmission Holdings Inc Common,Industrials
22,AVPT,NaN,1890.0,-1890.0,0.0,12.165048,NaN,NaN,NaN,NaN,NaN
12,CALM,position,510.0,20.0,530.0,88.647706,48245.9,0.07,81.19,Cal-Maine Foods Inc Common,Consumer Staples
17,CNK,position,620.0,60.0,680.0,26.857813,22018.4,0.03,25.36,Cinemark Holdings Inc Common,Communication Services
0,CNX,position,580.0,-20.0,560.0,35.514876,22327.2,0.03,29.58,CNX Resources Corp Common,Energy
15,CSWI,position,60.0,-10.0,50.0,352.598387,21150.0,0.03,325.21,CSW Industrials Inc Common,Industrials
19,DORM,old,0.0,150.0,150.0,NaN,20331.0,0.03,107.97,Dorman Products Inc Common,Consumer Discretionary
9,ENVA,position,460.0,-90.0,370.0,81.466790,38113.7,0.06,84.68,Enova International Inc Common,Financials


Peer closed connection.
